In [0]:
import numpy as np
import pandas as pd
import sklearn
import sklearn.linear_model as lm
from sklearn import preprocessing
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import metrics
import warnings
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve,auc,recall_score,precision_score,accuracy_score,f1_score
from sklearn.metrics import confusion_matrix,average_precision_score,recall_score
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [0]:
df=pd.read_excel("titanic.xls")
df.shape
df.info()

(1309, 14)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 14 columns):
pclass       1309 non-null int64
survived     1309 non-null int64
name         1309 non-null object
sex          1309 non-null object
age          1046 non-null float64
sibsp        1309 non-null int64
parch        1309 non-null int64
ticket       1309 non-null object
fare         1308 non-null float64
cabin        295 non-null object
embarked     1307 non-null object
boat         486 non-null object
body         121 non-null float64
home.dest    745 non-null object
dtypes: float64(3), int64(4), object(7)
memory usage: 143.3+ KB


In [0]:
df.head()
import statistics
titanic=df
#titanic=df.fillna({'Age':df['age'].median(),'embarked':statistics.mode})

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [0]:
titanic.drop(['name','boat','body','home.dest','ticket','cabin'],axis=1,inplace=True)

In [0]:
titanic.head()
titanic.rename(columns={'sex':'gender','sibsp':'siblings','parch':'parents_child'}, inplace=True)

,pclass,survived,sex,age,sibsp,parch,fare,embarked
0,1,1,female,29.0000,0,0,211.3375,S
1,1,1,male,0.9167,1,2,151.5500,S
2,1,0,female,2.0000,1,2,151.5500,S
3,1,0,male,30.0000,1,2,151.5500,S
4,1,0,female,25.0000,1,2,151.5500,S


In [0]:
titanic['embarked']=titanic['embarked'].fillna(titanic['embarked'].mode()[0])

In [0]:
titanic=titanic.fillna(titanic.median())
for x in titanic.columns:  
  if titanic[x].dtype=='object':
      titanic[x]=pd.Categorical(titanic[x]).codes
titanic.head()

,pclass,survived,gender,age,siblings,parents_child,fare,embarked
0,1,1,0,29.0000,0,0,211.3375,2
1,1,1,1,0.9167,1,2,151.5500,2
2,1,0,0,2.0000,1,2,151.5500,2
3,1,0,1,30.0000,1,2,151.5500,2
4,1,0,0,25.0000,1,2,151.5500,2


In [0]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 8 columns):
pclass           1309 non-null int64
survived         1309 non-null int64
gender           1309 non-null int8
age              1309 non-null float64
siblings         1309 non-null int64
parents_child    1309 non-null int64
fare             1309 non-null float64
embarked         1309 non-null int8
dtypes: float64(2), int64(4), int8(2)
memory usage: 64.0 KB


In [0]:
titanic=titanic[:-1]
titanic.info()
x=titanic.drop(['survived'],axis=1)
y=titanic['survived']
y.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1308 entries, 0 to 1307
Data columns (total 8 columns):
pclass           1308 non-null int64
survived         1308 non-null int64
gender           1308 non-null int8
age              1308 non-null float64
siblings         1308 non-null int64
parents_child    1308 non-null int64
fare             1308 non-null float64
embarked         1308 non-null int8
dtypes: float64(2), int64(4), int8(2)
memory usage: 64.0 KB


0    1
1    1
2    0
3    0
4    0
Name: survived, dtype: int64

In [0]:
x.head()

,pclass,gender,age,siblings,parents_child,fare,embarked
0,1,0,29.0000,0,0,211.3375,2
1,1,1,0.9167,1,2,151.5500,2
2,1,0,2.0000,1,2,151.5500,2
3,1,1,30.0000,1,2,151.5500,2
4,1,0,25.0000,1,2,151.5500,2


In [0]:
n_estimators=[10,500,100,2000]
criterion=['gini','entropy']
min_samples_split=[2,100,20,50,100]
max_depth=[20,30,40,50,None]

In [0]:
param_grid={'n_estimators':n_estimators,'criterion':criterion,'min_samples_split':min_samples_split,'max_depth':max_depth}
rf=RandomForestClassifier()
rf_g=RandomizedSearchCV(rf,param_grid,cv=5,verbose=2,n_jobs=1)
rf_g.fit(x,y)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] n_estimators=2000, min_samples_split=20, max_depth=40, criterion=entropy 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  n_estimators=2000, min_samples_split=20, max_depth=40, criterion=entropy, total=   4.4s
[CV] n_estimators=2000, min_samples_split=20, max_depth=40, criterion=entropy 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.4s remaining:    0.0s


[CV]  n_estimators=2000, min_samples_split=20, max_depth=40, criterion=entropy, total=   4.3s
[CV] n_estimators=2000, min_samples_split=20, max_depth=40, criterion=entropy 
[CV]  n_estimators=2000, min_samples_split=20, max_depth=40, criterion=entropy, total=   4.3s
[CV] n_estimators=2000, min_samples_split=20, max_depth=40, criterion=entropy 
[CV]  n_estimators=2000, min_samples_split=20, max_depth=40, criterion=entropy, total=   4.4s
[CV] n_estimators=2000, min_samples_split=20, max_depth=40, criterion=entropy 
[CV]  n_estimators=2000, min_samples_split=20, max_depth=40, criterion=entropy, total=   4.3s
[CV] n_estimators=2000, min_samples_split=20, max_depth=None, criterion=gini 
[CV]  n_estimators=2000, min_samples_split=20, max_depth=None, criterion=gini, total=   3.9s
[CV] n_estimators=2000, min_samples_split=20, max_depth=None, criterion=gini 
[CV]  n_estimators=2000, min_samples_split=20, max_depth=None, criterion=gini, total=   3.9s
[CV] n_estimators=2000, min_samples_split=20,

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:  1.3min finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [0]:
#PREDICT USING THE TRAINED MODEL
predicted_values=rf_g.predict(x)
predicted_values


array([1, 1, 1, ..., 0, 0, 0])

In [0]:
#ACCUIRACY SCORE
print("ACCUIRACY SCORE")
print(metrics.accuracy_score(y,predicted_values))

ACCUIRACY SCORE
0.8692660550458715


In [0]:
#PRECISION RECALL MATRIX
print("precision/recall Metrics")
print(metrics.classification_report(y,predicted_values))


precision/recall Metrics
              precision    recall  f1-score   support

           0       0.86      0.94      0.90       808
           1       0.89      0.75      0.81       500

    accuracy                           0.87      1308
   macro avg       0.87      0.85      0.86      1308
weighted avg       0.87      0.87      0.87      1308



In [0]:
rf_g.best_estimator_

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=20,
                       min_weight_fraction_leaf=0.0, n_estimators=2000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [0]:
rf_g.best_score_

0.6758298967564564

In [0]:
cv=pd.DataFrame(rf_g.cv_results_)

In [0]:
cv.to_csv("test.csv")

In [0]:
#we will also split our data into train and test datasets and Build the model on train dataset and apply the model on test datset.
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
import pydotplus
from sklearn.model_selection import GridSearchCV
#BUILDING THE MODEL
mytreeone=RandomForestClassifier()
#CROSS VALIDATION TELLS HOW A MODEL PERFORMS ON A DATASET USING MULTIPLE SAMPLES OF TRAIN DATA
mytreeone_g=GridSearchCV(mytreeone,param_grid,cv=5,verbose=2,n_jobs=-1)
mytreeone_g.fit(x,y)


Fitting 5 folds for each of 200 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   35.2s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 361 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done 644 tasks      | elapsed: 11.1min
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed: 17.7min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rando

In [0]:
mytreeone_g.best_estimator_

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=50, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=100,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [0]:
mytreeone_g.best_score_

0.7033254365183821

In [0]:
#PREDICT USING THE TRAINED MODEL
predicted_values=mytreeone_g.predict(x)
predicted_values

array([1, 1, 1, ..., 1, 0, 0])

In [0]:
#ACCUIRACY SCORE
print("ACCUIRACY SCORE")
print(metrics.accuracy_score(y,predicted_values))

ACCUIRACY SCORE
0.8165137614678899


In [0]:
#PRECISION RECALL MATRIX
print("precision/recall Metrics")
print(metrics.classification_report(y,predicted_values))

precision/recall Metrics
              precision    recall  f1-score   support

           0       0.83      0.88      0.86       808
           1       0.78      0.72      0.75       500

    accuracy                           0.82      1308
   macro avg       0.81      0.80      0.80      1308
weighted avg       0.81      0.82      0.81      1308

